In [ ]:
import tensorflow as tf

In [ ]:
import glob
import os

In [ ]:
import numpy as np

In [ ]:
import random

In [ ]:
from utils import *

In [ ]:
slim = tf.contrib.slim

In [ ]:
HEIGHT, WIDTH, CHANNEL = 64, 64, 3
BATCH_SIZE = 64

In [ ]:
random_dim = 128

In [ ]:
imgpath = 'RGB_emoji/'

In [ ]:
def lrelu(x, n, leak=0.2): 
    return tf.maximum(x, leak * x, name=n)

In [ ]:
def process_data():
    images = glob.glob(imgpath+'*.jpg')
    # print images    
    all_images = tf.convert_to_tensor(images, dtype = tf.string)
    
    images_queue = tf.train.slice_input_producer([all_images],shuffle=False)
                                        
    content = tf.read_file(images_queue[0])
    image = tf.image.decode_jpeg(content, channels = CHANNEL)
    
    size = [HEIGHT, WIDTH]
    image = tf.image.resize_images(image, size)
    image.set_shape([HEIGHT,WIDTH,CHANNEL])
    
    image = tf.cast(image, tf.float32)
    image = ((image / 255.0)-0.5)*2
    
    images_batch = tf.train.shuffle_batch(
                                    [image], batch_size = BATCH_SIZE,
                                    num_threads = 4, capacity = 200 + 3* BATCH_SIZE,
                                    min_after_dequeue = 200)
    num_images = len(images)

    return images_batch, num_images

In [ ]:
def generator(input, random_dim, is_train, reuse=False):
    c4, c8, c16, c32, c64 = 512, 256, 128, 64, 32 # channel num
    s4 = 4
    output_dim = CHANNEL  # RGB image
    with tf.variable_scope('gen') as scope:
        if reuse:
            scope.reuse_variables()
        w1 = tf.get_variable('w1', shape=[random_dim, s4 * s4 * c4], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('b1', shape=[c4 * s4 * s4], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        flat_conv1 = tf.add(tf.matmul(input, w1), b1, name='flat_conv1')

        conv1 = tf.reshape(flat_conv1, shape=[-1, s4, s4, c4], name='conv1') # 4*4*512
        bn1 = tf.contrib.layers.batch_norm(conv1, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn1')
        act1 = tf.nn.relu(bn1, name='act1')
        
        # 8*8*256
        #Convolution, bias, activation, repeat! 
        conv2 = tf.layers.conv2d_transpose(act1, c8, kernel_size=[3, 3], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn2')
        act2 = tf.nn.relu(bn2, name='act2')
        # 16*16*128
        conv3 = tf.layers.conv2d_transpose(act2, c16, kernel_size=[3, 3], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn3')
        act3 = tf.nn.relu(bn3, name='act3')
        # 32*32*64
        conv4 = tf.layers.conv2d_transpose(act3, c32, kernel_size=[3, 3], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn4')
        act4 = tf.nn.relu(bn4, name='act4')
        # 64*64*32
        conv5 = tf.layers.conv2d_transpose(act4, c64, kernel_size=[3, 3], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv5')
        bn5 = tf.contrib.layers.batch_norm(conv5, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn5')
        act5 = tf.nn.relu(bn5, name='act5')
        
        #64*64*16
        conv6 = tf.layers.conv2d_transpose(act5, 16, kernel_size=[3, 3], strides=[1, 1], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv6')
        bn6 = tf.contrib.layers.batch_norm(conv6, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn6')
        act6 = tf.nn.relu(bn6, name='act6')
        
        #64*64*8
        conv7 = tf.layers.conv2d_transpose(act6, 8, kernel_size=[3, 3], strides=[1, 1], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv7')
        bn7 = tf.contrib.layers.batch_norm(conv7, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn7')
        act7 = tf.nn.relu(bn7, name='act7')
        
        #64*64*8
        conv8 = tf.layers.conv2d_transpose(act7, 8, kernel_size=[3, 3], strides=[1, 1], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv8')
        bn8 = tf.contrib.layers.batch_norm(conv8, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn8')
        act8 = tf.nn.relu(bn8, name='act8')
        
        #64*64*3
        conv9 = tf.layers.conv2d_transpose(act8, 3, kernel_size=[3, 3], strides=[1, 1], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv9')
        
        
        
        act9 = tf.nn.tanh(conv9, name='act9')
        return act9

In [ ]:
def discriminator(input, is_train, reuse=False):
    c1,c2, c4, c8, c16 = 32, 64, 128, 256, 512  # channel num: 32,64, 128, 256, 512
    with tf.variable_scope('dis') as scope:
        if reuse:
            scope.reuse_variables()
            
        #Convolution, activation, bias, repeat! 
        conv0 = tf.layers.conv2d(input, c1, kernel_size=[3, 3], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv0')
        ln0 = tf.contrib.layers.layer_norm(conv0, center=True, scale=True, scope = 'ln0')
        act0 = lrelu(ln0, n='act0')
        
        #Convolution, activation, bias, repeat! 
        conv1 = tf.layers.conv2d(act0, c2, kernel_size=[3, 3], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv1')
        ln1 = tf.contrib.layers.layer_norm(conv1, center=True, scale=True, scope = 'ln1')
        act1 = lrelu(ln1, n='act1')
         #Convolution, activation, bias, repeat! 
        conv2 = tf.layers.conv2d(act1, c4, kernel_size=[3, 3], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv2')
        ln2 = tf.contrib.layers.layer_norm(conv2, center=True, scale=True, scope='ln2')
        act2 = lrelu(ln2, n='act2')
        #Convolution, activation, bias, repeat! 
        conv3 = tf.layers.conv2d(act2, c8, kernel_size=[3, 3], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv3')
        ln3 = tf.contrib.layers.layer_norm(conv3, center=True, scale=True, scope='ln3')
        act3 = lrelu(ln3, n='act3')
         #Convolution, activation, bias, repeat! 
        conv4 = tf.layers.conv2d(act3, c16, kernel_size=[3, 3], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv4')
        ln4 = tf.contrib.layers.layer_norm(conv4,  center=True, scale=True, scope='ln4')
        act4 = lrelu(ln4, n='act4')
               
        # start from act4
        dim = int(np.prod(act4.get_shape()[1:]))
        fc1 = tf.reshape(act4, shape=[-1, dim], name='fc1')
        
        w2 = tf.get_variable('w2', shape=[fc1.shape[-1], 1], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b2 = tf.get_variable('b2', shape=[1], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))

        # wgan just get rid of the sigmoid
        logits = tf.add(tf.matmul(fc1, w2), b2, name='logits')

        
        return logits

In [ ]:
with tf.variable_scope('input'):
    #real and fake image placholders
    real_image = tf.placeholder(tf.float32, shape = [None, HEIGHT, WIDTH, CHANNEL], name='real_image')
    random_input = tf.placeholder(tf.float32, shape=[None, random_dim], name='rand_input')
    is_train = tf.placeholder(tf.bool, name='is_train')

In [ ]:
# wgan
fake_image = generator(random_input, random_dim, is_train)
    
real_result = discriminator(real_image, is_train)
fake_result = discriminator(fake_image, is_train, reuse=True)

In [ ]:
d_loss = tf.reduce_mean(fake_result) - tf.reduce_mean(real_result)  # This optimizes the discriminator.
g_loss = -tf.reduce_mean(fake_result)  # This optimizes the generator.

In [ ]:
# gradient penalty
LAMBDA = 500

alpha = tf.random_uniform(
        shape=[], 
        minval=0.,
        maxval=1.
)
differences = fake_image - real_image
interpolates = real_image + tf.multiply(alpha,differences)
gradients = tf.gradients(discriminator(interpolates,is_train, reuse=True), [interpolates])[0]
slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=1))

gradient_penalty = tf.reduce_mean((slopes-1.)**2)
d_loss += LAMBDA*gradient_penalty

In [ ]:
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'dis' in var.name]
g_vars = [var for var in t_vars if 'gen' in var.name]

In [ ]:
trainer_d = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(d_loss, var_list=d_vars)
trainer_g = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(g_loss, var_list=g_vars)

In [ ]:
batch_size = BATCH_SIZE
image_batch, samples_num = process_data()

In [ ]:
batch_num = int(samples_num / batch_size)

In [ ]:
save_path = 'model/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
log_path = 'logs/'
if not os.path.exists(log_path):
    os.makedirs(log_path)

In [ ]:
example_path = 'example/'
if not os.path.exists(example_path):
    os.makedirs(example_path)

In [ ]:
EPOCH = 90000

In [ ]:
d_iters = 5
g_iters = 1

In [ ]:
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    writer_d_loss = tf.summary.FileWriter(log_path+'/d_loss',sess.graph)
    writer_g_loss = tf.summary.FileWriter(log_path+'/g_loss')

    loss_var = tf.Variable(0.0)
    tf.summary.scalar("loss", loss_var)
    write_op = tf.summary.merge_all()
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    start = 0
    ckpt = tf.train.latest_checkpoint(save_path)
    if ckpt:
        print 'restore:'+str(ckpt)
        saver.restore(sess, ckpt)
        start = int(ckpt.split('-')[-1])+1
        
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    for step in range(start,EPOCH*batch_num):
        train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
        
        train_image = sess.run(image_batch)
        
        for k in range(d_iters):
            _, dLoss = sess.run([trainer_d, d_loss],
                                    feed_dict={random_input: train_noise, real_image: train_image, is_train: True})
        if d_iters!=0:
            print 'dLoss:'+str(dLoss)


        for k in range(g_iters):
            _, gLoss = sess.run([trainer_g, g_loss],
                                    feed_dict={random_input: train_noise, is_train: True})
        if g_iters!=0:
            print 'gLoss:'+str(gLoss)
        
            
                
        if step%25 == 0:
            saver.save(sess, save_path + 'model.ckpt', global_step=step)
            
            if d_iters!=0:
                summary = sess.run(write_op, {loss_var: dLoss})
                writer_d_loss.add_summary(summary, step)
                writer_d_loss.flush()
            if g_iters!=0:
                summary = sess.run(write_op, {loss_var: gLoss})
                writer_g_loss.add_summary(summary, step)
                writer_g_loss.flush()
            
        if step%25 == 0:
            # sample_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
            imgtest = sess.run(fake_image, feed_dict={random_input: train_noise, is_train: False})
            save_images(imgtest, [8,8] , example_path+'/step_' + str(step) + '.jpg')
    
    coord.request_stop()
    coord.join(threads)

In [ ]:
def test():
    random_dim = 128
    gen_path = 'testgen/'
    if not os.path.exists(gen_path):
        os.makedirs(gen_path)
    
    tf.reset_default_graph()
    with tf.variable_scope('input'):
        #real and fake image placholders
        real_image = tf.placeholder(tf.float32, shape = [None, HEIGHT, WIDTH, CHANNEL], name='real_image')
        random_input = tf.placeholder(tf.float32, shape=[None, random_dim], name='rand_input')
        is_train = tf.placeholder(tf.bool, name='is_train')
    # wgan
    fake_image = generator(random_input, random_dim, is_train)

    real_result = discriminator(real_image, is_train)
    fake_result = discriminator(fake_image, is_train, reuse=True)
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        ckpt = tf.train.latest_checkpoint('model/')
        saver.restore(sess, ckpt)
        
        sample_noise1 = np.random.uniform(-1.0, 1.0, size=[1, random_dim]).astype(np.float32)
        imgtest = sess.run(fake_image, feed_dict={random_input: sample_noise1, is_train: False})
        save_images(imgtest, [1,1] , gen_path+'/example1.jpg')
        
        sample_noise2 = np.random.uniform(-1.0, 1.0, size=[1, random_dim]).astype(np.float32)
        imgtest = sess.run(fake_image, feed_dict={random_input: sample_noise2, is_train: False})
        save_images(imgtest, [1,1] , gen_path+'/example2.jpg')
        
        sample_noise3 = np.random.uniform(-1.0, 1.0, size=[1, random_dim]).astype(np.float32)
        imgtest = sess.run(fake_image, feed_dict={random_input: sample_noise3, is_train: False})
        save_images(imgtest, [1,1] , gen_path+'/example3.jpg')
        
        sample_noise4 = sample_noise1 - sample_noise2 + sample_noise3
        imgtest = sess.run(fake_image, feed_dict={random_input: sample_noise4, is_train: False})
        save_images(imgtest, [1,1] , gen_path+'/example4.jpg')

In [ ]:
test()